## 步骤

此笔记本展示如何使用 Amazon Personalize 的新的 User Personalization 配方（aws-user-personalization）。此配方平衡新旧项目之间的推荐，允许您调整平衡以支持更多新项目或旧项目

概括来说，使用新的 USER Personalization 配方涉及以下步骤：

1. 设置 Personalize 客户端
2. 创建数据集组、定义架构、导入数据集并提取实时交互
4. 使用新配置 `campaignConfig` 创建活动
5. 创建事件跟踪器以提取由 PutEvents 发送的事件。
6. 调用 GetRecommendations，响应中会返回一个新字段 `RecommendationId`。
7. 使用 `RecommendationId` 或自定义展示项目列表调用 putEvents。
8. 等待活动更新。
9. 更新活动，停止自动更新。
10. 清理


> **注意**：**执行此笔记本需要几个小时。**

### 用于 AWS API 访问的访问密钥/私人密钥设置。

确保您使用的访问密钥、私人密钥具有相应的权限。还要选择运行此演示的区域

In [ ]:
accessKeyId = ""
secretAccessKey = ""
region_name = ""

In [ ]:
import os
import boto3
from botocore.exceptions import ClientError
import time
import numpy as np
import pandas as pd
import json
from datetime import datetime

In [ ]:
suffix = str(np.random.uniform())[4:9]
prefix = 'user-personalization-'
print('prefix+suffix:{}{}'.format(prefix, suffix))
s3_bucket_name = (prefix + suffix).lower()
interaction_schema_name = prefix + 'interaction-'  + suffix
item_metadata_schema_name = prefix + 'items-'  + suffix
dataset_group_name = prefix + suffix
interaction_dataset_name = prefix + 'interactions-' + suffix
item_metadata_dataset_name = prefix + 'items-' + suffix
event_tracker_name = prefix + suffix
solution_name = prefix + suffix
event_tracker_name = prefix + suffix
campaign_name = prefix + suffix

### 1. 客户端设置
让我们首先为 Personalize 和 S3 设置客户端。

In [ ]:
# Public s3 bucket owned by Personalize service which used to store the example dataset.

personalize_s3_bucket = "personalize-cli-json-models"
s3_client = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('s3')



####  初始化 Personalize 客户端

In [ ]:

personalize = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize')
personalize_runtime = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-runtime')
personalize_events = boto3.Session(aws_access_key_id=accessKeyId,
                           aws_secret_access_key=secretAccessKey, region_name=region_name).client('personalize-events')

### 样本数据集

为方便起见，并出于本演示的目的，我们将使用 Personalize 提供的示例数据集。
我们提供了两个数据集，一个是项目元数据，另一个是交互数据集。我们先将数据集下载到本地

In [ ]:
interaction_dataset_key = "sample-dataset/interactions-sample.csv"
items_dataset_key = "sample-dataset/items-with-creation-timestamp-sample.csv"
interactions_file = os.getcwd() + "/interaction_raw.csv"
items_metadata_file = os.getcwd() + "/items_raw.csv"
s3_client.download_file(personalize_s3_bucket, interaction_dataset_key, interactions_file)
s3_client.download_file(personalize_s3_bucket, items_dataset_key, items_metadata_file)

In [ ]:
interactions_df = pd.read_csv(interactions_file)
items_df = pd.read_csv(items_metadata_file)

#### 让我们浏览一下交互数据框

In [ ]:
interactions_df.head(2)

#### 交互数据集
**ITEM_ID**：与 EVENT_TYPE 对应的项目。

**EVENT_TYPE**：事件类型。

**TIMESTAMP**：交互的时间戳（毫秒）。（注意是以毫秒为单位） 

**USER_ID**：与此展示对应的用户 ID。

**IMPRESSION：**现在，您可以选择将展示数据与交互数据集中的事件数据一起传递。正如您在上面所看到的，这是在一个新的字段 `IMPRESSION` 中传递的，该字段采用管道连接用户与之交互的项目（例如，显示给用户的项目），展示还包括点击的项目。


In [ ]:
items_df.head(2)

#### 项目数据集

项目数据集包含 Item_ids 和关联的元数据。

**ITEM_ID**：项目 ID  

**流派** 项目的元数据，如果同一项目有多个类别分类，则使用流水线 " | " 连接。

**creation_timestamp** 添加项目时的时间戳

有关数据集的更多详细信息可以在以下文档中找到：
https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html

#### 更新时间戳

在这里，我们将把数据集的时间戳更新为从今天开始的 8 天，以显示实时交互对我们推荐的影响

In [ ]:
current_time = int(time.time())
one_hour_ago = current_time - 8 * 24 * 60 * 60
# Get the time gap between the latest timestamp in the interaction and the current time 
interactions_df = interactions_df.astype({"TIMESTAMP": 'int64'})
latest_time_in_csv = interactions_df["TIMESTAMP"].max()
delta = one_hour_ago - latest_time_in_csv

In [ ]:
# shift the latest timestamp in the interactions_df to be the last hour timestamp
interactions_df.TIMESTAMP = interactions_df.TIMESTAMP + delta
interactions_df.to_csv(os.getcwd() + "/interaction.csv", index = False)

# shift the latest timestamp in the items_df to be the last hour timestamp
items_df = items_df.astype({"creation_timestamp": 'int64'})
items_df.creation_timestamp = items_df.creation_timestamp + delta
items_df.to_csv(os.getcwd() + "/items.csv", index = False)

#### 更新后，让我们再次检查交互和项目数据集

In [ ]:
interactions_df.head(2)

In [ ]:
items_df.head(2)


### 2. 将数据提取到 Amazon Personalize
现在，让我们创建一个数据集组、创建架构、上传数据集、创建一个 datasetImport 作业。这一点没有改变。

#### a. 创建数据集组

这类似于现有配方，完整文档可在[此处](https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html)找到

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)
dataset_group_arn = create_dataset_group_response['datasetGroupArn']

In [ ]:

print('dataset_group_arn : {}'.format(dataset_group_arn))

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

#### b. 创建数据集架构

> **_注意_**：`Impression` 字段具有类型字符串，并对多个值使用管道连接。

In [ ]:
interaction_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "IMPRESSION",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
    ],
    "version": "1.0"
}

In [ ]:
interaction_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(interaction_schema)
)
# print(json.dumps(create_schema_response, indent=2))
interaction_schema_arn = interaction_schema_response['schemaArn']
print('interaction_schema_arn:\n', interaction_schema_arn)

In [ ]:
item_metadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRES",
        "type": "string",
        "categorical": True
    },
    {
        "name": "CREATION_TIMESTAMP",
        "type": "long"
    }
    ],
    "version": "1.0"
}

item_metadata_schema_response = personalize.create_schema(
    name = item_metadata_schema_name,
    schema = json.dumps(item_metadata_schema)
)

# print(json.dumps(create_schema_response, indent=2))
item_metadata_schema_arn = item_metadata_schema_response['schemaArn']
print('item_metadata_schema_arn:\n', item_metadata_schema_arn)

#### c. 创建数据集

In [ ]:
interactions_dataset_response = personalize.create_dataset(
    datasetType = 'INTERACTIONS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn,
    name = interaction_dataset_name
)
interaction_dataset_arn = interactions_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('interaction_dataset_arn:\n', interaction_dataset_arn)

items_dataset_response = personalize.create_dataset(
    datasetType = 'ITEMS',
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_metadata_schema_arn,
    name = item_metadata_dataset_name
)
item_metadata_dataset_arn = items_dataset_response['datasetArn']
#print(json.dumps(create_dataset_response, indent=2))
print('item_metadata_dataset_arn:\n', item_metadata_dataset_arn)

#### d. 将数据集上载到 S3 存储桶，设置相应的 S3 存储桶策略、IAM 角色等。

我们需要上载这些数据集，您也可以提供已有数据集的存储桶名称

In [ ]:
#!aws s3 mb s3://{s3_bucket_name}
s3_bucket_name

In [ ]:
s3_client.create_bucket(Bucket=s3_bucket_name,
                          CreateBucketConfiguration={
                              'LocationConstraint': region_name})

In [ ]:
interactions_file = os.getcwd() + "/interaction.csv"
items_metadata_file = os.getcwd() + "/items.csv"

In [ ]:
s3_client.upload_file(Filename=interactions_file, Bucket=s3_bucket_name,
    Key="interaction.csv")
s3_client.upload_file(Filename=items_metadata_file, Bucket=s3_bucket_name,
    Key="items.csv")

#### e. 将策略附加到您的 S3 存储桶

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(s3_bucket_name),
                "arn:aws:s3:::{}/*".format(s3_bucket_name)
            ]
        }
    ]
}

s3_client.put_bucket_policy(Bucket=s3_bucket_name, Policy=json.dumps(policy));

#### f. 设置相应的 IAM 角色，以便 Personalize 可以访问数据集

In [ ]:
iam = boto3.client(service_name='iam', 
                         aws_access_key_id = accessKeyId, 
                         aws_secret_access_key = secretAccessKey)   


role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

In [ ]:
print('role_arn:', role_arn)

#### g. 创建 DataSetImportJobs 以上载数据

In [ ]:
time.sleep(20) # wait for RoleARN completion
interactions_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'interactions-dij-' + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'interaction.csv')
    },
    roleArn = role_arn
)

interactions_dij_arn = interactions_dij_response['datasetImportJobArn']
print('interactions_dij_arn: ', interactions_dij_arn)
#print(json.dumps(interactions_dij_arn, indent=2))

items_dij_response = personalize.create_dataset_import_job(
    jobName =  prefix + 'items-dij-' + suffix,
    datasetArn = item_metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(s3_bucket_name, 'items.csv')
    },
    roleArn = role_arn
)

items_dij_arn = items_dij_response['datasetImportJobArn']
print('items_dij_arn:', items_dij_arn)

In [ ]:
dataset_job_arns = [interactions_dij_arn, items_dij_arn]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time and len(dataset_job_arns) != 0:
    time.sleep(60)    
    for dij_arn in dataset_job_arns:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dij_arn
        )
        dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
        status = None
        if "latestDatasetImportJobRun" not in dataset_import_job:
            status = dataset_import_job["status"]
            print("{} : {}".format(dij_arn, status))
        else:
            status = dataset_import_job["latestDatasetImportJobRun"]["status"]
            print("DIJ_ARN: {}, LatestDatasetImportJobRun: {}".format(dij_arn, status))
    
        if status == "ACTIVE" or status == "CREATE FAILED":
            dataset_job_arns.remove(dij_arn)


### 3. 创建解决方案、解决方案版本

我们将使用“aws-user-personalization”创建解决方案。该配方平衡交付给用户的新旧项目的推荐

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"
max_time = time.time() + 3*60*60 # 3 hours
create_solution_response = None
while time.time() < max_time:

    try:
        create_solution_response = personalize.create_solution(name=solution_name, 
                                    recipeArn= recipe_arn, 
                                    datasetGroupArn = dataset_group_arn)

        solution_arn = create_solution_response['solutionArn']
        print('solution_arn: ', solution_arn)
        break;
    except personalize.exceptions.ClientError as e:
        if 'EVENT_INTERACTIONS' not in str(e):
            print(json.dumps(create_solution_response, indent=2))
            print(e)
            break

#### 创建解决方案版本

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 4. 创建活动
在创建活动时，我们可以设置 itemExplorationConfig 以配置冷项目探索权重和探索年龄截止值。现在，我们可以将 explorationWeight 设置为更高的 0.9，将 explorationItemAgeCutOff 设置为 7，因此我们认为所有创建时间少于 7 天的项目都将被视为冷项目，我们将对这些新项目进行更多的探索。

#### 创建活动

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = prefix + suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.9",
            "explorationItemAgeCutOff": "7"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
describe_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)
campaign_summary = describe_campaign_response["campaign"]
campaign_summary

### 5. 调用 GetRecommendations
出于演示目的，我们将使用输入数据集中的用户 ID 来进行 getRecommendation 调用。
> **_注意_**：在响应中，您有一个新字段 `RecommendationId`，它对应于 Personalize GetRecommendations 返回的项目列表。您可以传递此 RecommendationId 以指示展示。
您还可以将展示作为项目的管道字符串连接进行传递，如果您同时传递 RecommendationId 和 ImpressionList，则 ImpressionList 优先并在系统中使用。

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')
print(rec_response['recommendationId'])

In [ ]:
rec_response['itemList']

### 6. 创建事件跟踪器

创建一个在使用 PutEvents API 将事件数据发送到指定的数据集组时使用的事件跟踪器。

In [ ]:
even_tracker_response = personalize.create_event_tracker( 
    name=event_tracker_name,
    datasetGroupArn=dataset_group_arn
)
event_tracker_arn  = even_tracker_response['eventTrackerArn']
event_tracking_id = even_tracker_response['trackingId']
#print(json.dumps(even_tracker_response,indent=2))
print('eventTrackerArn:{},\n eventTrackingId:{}'.format(event_tracker_arn, event_tracking_id))

### 7. 通过 PutEvents 发送展示数据以进行个性化设置。
Amazon Personalize 可以对两种展示类型进行建模：
1. 隐性展示和显性展示。隐性展示是在用户会话期间出现的展示，每当向用户显示某个项目时，Amazon Personalize 都会自动记录这些展示。您可以将这些展示集成到您的推荐工作流中，方法是将 RecommendationID（由 and 操作返回）作为未来 PutEvents 请求的输入。


2. 显性展示是您在发出 PutEvents 请求时手动输入的展示。例如，当您由于无法使用等原因而没有显示 GetRecommendations 返回的某些项目时，您可以使用显性展示。

> **注意**：如果您在上述交互架构中定义了 `impression`，则需要发送展示列表（从 GetRecommendations 返回的项目或您自己的项目）。
**当 recommendationId 和展示都要发送时，Amazon Personalize 将默认使用显性展示。**



#### 让我们把以前推荐的项目作为展示

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
     'sentAt': datetime.now().timestamp(),
     'eventType' : 'click',
     'itemId' : rec_response['itemList'][0]['itemId'],        
     'recommendationId': rec_response['recommendationId'],
     'impression': [item['itemId'] for item in rec_response['itemList']],
     }]
    )

#### 我们也可以放入一些新项目

让我们将新的 itemId“2xx”放入 Personalize。

In [ ]:
personalize_events.put_events(
     trackingId = event_tracking_id,
     userId= '101',
     sessionId = '1',
     eventList = [{
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '240',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '241',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '242',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '243',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '244',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '245',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '246',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '247',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '248',
     },
     {
         'sentAt': datetime.now().timestamp(),
         'eventType' : 'click',
         'itemId' : '249',
     }]
    )

### 8. 使用 updateMode 创建新的 SolutionVersion

放置事件后，请等待大约 15 分钟，以便 Personalize 提取新数据，然后使用更新模式创建新的 solutionVersion

In [ ]:
create_solution_version_response = personalize.create_solution_version(solutionArn = solution_arn, trainingMode = "UPDATE")

solution_version_after_update = create_solution_version_response['solutionVersionArn']
print('solution_version_after_update:', solution_version_arn)

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_after_update
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### 9. 更新活动

使用更新中的最新解决方案版本 arn 更新活动。

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, solutionVersionArn=solution_version_after_update)
print('campaign_arn_response: ', campaign_arn_response)

In [ ]:
# Wait for campaign update to reflect the new solution-version
solutionVersionArn = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    solutionVersionArn = describe_campaign_response["campaign"]["solutionVersionArn"]
    print("Campaign solution version: {}".format(solutionVersionArn))
    
    if solutionVersionArn == solution_version_after_update:
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']["solutionVersionArn"]
desc_campaign_response

### 更新解决方案版本后，我们再做一次推荐

我们希望在推荐中显示新的项目，因为我们将高探索设置为 0.9

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response['itemList']

### 使用不同的 explorationWeight 更新活动

由于我们设置了较低的 explorationWeight，我们希望在推荐列表中显示更多旧项目。

In [ ]:
desc_campaign_response = personalize.describe_campaign(campaignArn = campaign_arn)['campaign']
desc_campaign_response

In [ ]:
campaign_arn_response = personalize.update_campaign(campaignArn=campaign_arn, campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.1",
            "explorationItemAgeCutOff": "7"
        }
    })


In [ ]:
# Wait for campaign update to reflect the new explorationWeight
explorationWeight = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    explorationWeight = describe_campaign_response["campaign"]["campaignConfig"]['itemExplorationConfig']['explorationWeight']
    print("Current Campaign  explorationWeight: {}".format(explorationWeight))
    
    if explorationWeight == "0.1":
        break
        
    time.sleep(60)

# wait 1 minutes
time.sleep(60)

### 更新 explorationWeight 后

让我们再做一次推荐，我们应该在这里看到更多旧项目。

In [ ]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn, userId = '101')

In [ ]:
rec_response

#### 10. 删除资源

创建所有资源后，让我们清理所有资源。

In [ ]:
personalize.delete_campaign(campaignArn=campaign_arn)
while len(personalize.list_campaigns(solutionArn=solution_arn)['campaigns']):
    time.sleep(5)

personalize.delete_solution(solutionArn=solution_arn)
while len(personalize.list_solutions(datasetGroupArn=dataset_group_arn)['solutions']):
    time.sleep(5)

for dataset in personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']:
    personalize.delete_dataset(datasetArn=dataset['datasetArn'])
while len(personalize.list_datasets(datasetGroupArn=dataset_group_arn)['datasets']):
    time.sleep(5)
    
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)